In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    memory_key="summary",
    max_token_limit=100,
    return_messages=True,
    verbose=True,
)


examples = [
    {
        "movie": "Interstellar",
        "answer": "🚀🕳️🌌"
    },
    {
        "movie": "Frozen",
        "answer": "❄️👭🎶"
    },
    {
        "movie": "Joker",
        "answer": "🎭🃏🧨"
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What emoji best represent the {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert. You only reply with three emojis representing the movie."),
        example_prompt,
        MessagesPlaceholder(variable_name="summary"),
        ("human", "What do you know about {movie}?"),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["summary"]
    

chain = RunnablePassthrough.assign(summary=load_memory) | final_prompt | chat

def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)



/var/folders/bb/hwq64p2n5rl8w8dygq6q1xjh0000gn/T/ipykernel_34175/1853161776.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(
/var/folders/bb/hwq64p2n5rl8w8dygq6q1xjh0000gn/T/ipykernel_34175/1853161776.py:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [3]:

invoke_chain("Moana")

🌺🌊🚣‍♀️content='🌺🌊🚣\u200d♀️' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-3f48823b-e7a1-48bd-a688-c013d71cb161-0'


In [2]:

invoke_chain("Titanic")

🚢💔🌊content='🚢💔🌊' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-ea6e9195-5862-422d-a47a-de08b97eee7e-0'


In [4]:
memory_test_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie memory bot. You only reply with the title of the first movie asked."),
    MessagesPlaceholder(variable_name="summary"),
    ("human",  "What was the first movie I asked about?")
])
memory_test_chain = RunnablePassthrough.assign(summary=load_memory) | memory_test_prompt | chat

result = memory_test_chain.invoke({})


Titanic